In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#installing packages
!pip install -r requirements.txt 

# Start Running Cells from Here

In [38]:
import numpy as np
import pandas as pd

import math

from nltk.corpus import stopwords
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [3]:
article_df=pd.read_csv('/kaggle/input/articles-sharing-reading-from-cit-deskdrop/shared_articles.csv')
interactions_df=pd.read_csv('/kaggle/input/articles-sharing-reading-from-cit-deskdrop/users_interactions.csv')

In [4]:
article_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


## Going through the shared_articles dataset 

In [5]:
article_df.shape , interactions_df.shape 

((3122, 13), (72312, 8))

In [6]:
article_df.dtypes

timestamp           int64
eventType          object
contentId           int64
authorPersonId      int64
authorSessionId     int64
authorUserAgent    object
authorRegion       object
authorCountry      object
contentType        object
url                object
title              object
text               object
lang               object
dtype: object

In [7]:
for col in article_df.columns:
    if article_df[col].dtype == 'object':
        print(col,end='')
        if article_df[col].nunique() < 100:
            print(':',article_df[col].unique(),end='')
        
        print('\n')

eventType: ['CONTENT REMOVED' 'CONTENT SHARED']

authorUserAgent

authorRegion: [nan 'MG' 'ON' 'SP' 'GA' 'NY' 'IL' 'NSW' 'RI' 'NJ' 'RJ' 'TX' 'AB' 'FL'
 '13' 'WA' '?' 'CA' 'SC' 'MS']

authorCountry: [nan 'BR' 'CA' 'US' 'AU' 'PT']

contentType: ['HTML' 'RICH' 'VIDEO']

url

title

text

lang: ['en' 'pt' 'es' 'la' 'ja']



In [8]:
article_df.isna().sum()

timestamp             0
eventType             0
contentId             0
authorPersonId        0
authorSessionId       0
authorUserAgent    2442
authorRegion       2442
authorCountry      2442
contentType           0
url                   0
title                 0
text                  0
lang                  0
dtype: int64

### Cleaning the dataset

In [9]:
cleaned_article = article_df[article_df['eventType']=='CONTENT SHARED']

In [10]:
cleaned_article.shape, article_df.shape 

((3047, 13), (3122, 13))

## Going through the users_interactions dataset 

In [11]:
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [12]:
interactions_df.dtypes

timestamp       int64
eventType      object
contentId       int64
personId        int64
sessionId       int64
userAgent      object
userRegion     object
userCountry    object
dtype: object

In [13]:
for col in interactions_df.columns:
    if interactions_df[col].dtype == 'object':
        print(col,end='')
        if interactions_df[col].nunique() < 100:
            print(':',interactions_df[col].unique(),end='')
        
        print('\n')

eventType: ['VIEW' 'FOLLOW' 'BOOKMARK' 'LIKE' 'COMMENT CREATED']

userAgent

userRegion: [nan 'NY' 'MG' 'SP' 'RJ' 'GA' 'CA' '?' 'NJ' 'TX' '49' 'MH' 'D' 'PA' 'VA'
 'BE' 'AZ' 'IL' 'MI' 'FL' 'ES' 'BW' 'MA' 'PR' 'BC' 'NSW' 'DF' 'C' 'HE'
 'ON' 'MS' '13' '11' 'DC' 'NE' 'NC' '33' 'CO' 'LA' 'ENG' 'GO' '14' 'IN'
 'CT' '45' '27' 'WA' 'PE' 'MO' 'MN' 'RS' 'NH' '41' 'SC' 'BA' 'WI' 'RM'
 'OK' 'OH' 'CE' 'RI' 'AB' 'R' 'BOL' 'NV' 'OR' 'KS' 'ZH' 'MT' 'QLD' 'PI'
 'CL']

userCountry: [nan 'US' 'BR' 'JP' 'KR' 'IN' 'IE' 'SG' 'DE' 'CA' 'IS' 'AU' 'ZZ' 'ES' 'CN'
 'GB' 'MY' 'AR' 'NL' 'CL' 'IT' 'CO' 'PT' 'CH']



In [14]:
interactions_df.isna().sum()

timestamp          0
eventType          0
contentId          0
personId           0
sessionId          0
userAgent      15394
userRegion     15405
userCountry    15394
dtype: int64

In [15]:
#adding a new columns event_strength 
event_strength = {'VIEW': 1.0, 'FOLLOW':2.0, 'BOOKMARK':3.0,
                  'LIKE':4.0, 'COMMENT CREATED':5.0}
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_strength[x])

In [16]:
interactions_df[['eventType','eventStrength']]

,eventType,eventStrength
0,VIEW,1.0
1,VIEW,1.0
2,VIEW,1.0
3,FOLLOW,2.0
4,VIEW,1.0
...,...,...
72307,LIKE,4.0
72308,VIEW,1.0
72309,VIEW,1.0
72310,VIEW,1.0


In [17]:

interactions_df.groupby(['personId', 'contentId']).size()

personId              contentId           
-9223121837663643404  -8949113594875411859    1
                      -8377626164558006982    1
                      -8208801367848627943    1
                      -8187220755213888616    1
                      -7423191370472335463    8
                                             ..
 9210530975708218054   8477804012624580461    4
                       8526042588044002101    1
                       8856169137131817223    1
                       8869347744613364434    1
                       9209886322932807692    1
Length: 40710, dtype: int64

In [18]:
users_count = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
users_count

personId
-9223121837663643404    43
-9212075797126931087     5
-9207251133131336884     7
-9199575329909162940    11
-9196668942822132778     7
                        ..
 9165571805999894845     5
 9187866633451383747    17
 9191849144618614467    12
 9199170757466086545    14
 9210530975708218054    63
Length: 1895, dtype: int64

In [19]:
active_users = users_count[users_count>=5].copy()
active_users

personId
-9223121837663643404    43
-9212075797126931087     5
-9207251133131336884     7
-9199575329909162940    11
-9196668942822132778     7
                        ..
 9165571805999894845     5
 9187866633451383747    17
 9191849144618614467    12
 9199170757466086545    14
 9210530975708218054    63
Length: 1140, dtype: int64

In [20]:
active_users_Id = active_users.reset_index()[['personId']]
active_users_Id

,personId
0,-9223121837663643404
1,-9212075797126931087
2,-9207251133131336884
3,-9199575329909162940
4,-9196668942822132778
...,...
1135,9165571805999894845
1136,9187866633451383747
1137,9191849144618614467
1138,9199170757466086545


In [21]:
interactions_active_users = interactions_df.merge(active_users_Id, how='right', 
                                                  left_on='personId', right_on='personId')

In [22]:
interactions_active_users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventStrength
0,1463138398,VIEW,7516228655554309785,-9223121837663643404,-4482197405545551645,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
1,1463656314,VIEW,3041906492387035176,-9223121837663643404,-7824685088995468735,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
2,1464113091,VIEW,-3750879736572068916,-9223121837663643404,-2774275024909061125,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
3,1462283851,VIEW,-730957269757756529,-9223121837663643404,2625340673871268120,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
4,1462452127,VIEW,-8949113594875411859,-9223121837663643404,-3673331845456357462,Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKi...,SP,BR,1.0
...,...,...,...,...,...,...,...,...,...
69863,1482932930,VIEW,-1572252285162838958,9210530975708218054,-7673376373070890422,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
69864,1484126240,VIEW,3660989387512978561,9210530975708218054,1216354488778990261,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
69865,1483015651,VIEW,-1572252285162838958,9210530975708218054,-2172456475209001768,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
69866,1483015617,VIEW,-1572252285162838958,9210530975708218054,-2172456475209001768,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0


In [23]:
interactions_active_users.groupby(['eventType', 'eventStrength']).size()

eventType        eventStrength
BOOKMARK         3.0               2420
COMMENT CREATED  5.0               1579
FOLLOW           2.0               1379
LIKE             4.0               5625
VIEW             1.0              58865
dtype: int64

### Applying log transformation to smooth the distribution

In [24]:
def smooth_user_preference(x):
    return math.log(x+1, 2)

In [25]:
interactions_full = interactions_active_users.groupby(['personId', 'contentId'])['eventStrength'].sum().apply(smooth_user_preference).reset_index()
                                        

interactions_full

,personId,contentId,eventStrength
0,-9223121837663643404,-8949113594875411859,1.000000
1,-9223121837663643404,-8377626164558006982,1.000000
2,-9223121837663643404,-8208801367848627943,1.000000
3,-9223121837663643404,-8187220755213888616,1.000000
4,-9223121837663643404,-7423191370472335463,3.169925
...,...,...,...
39101,9210530975708218054,8477804012624580461,3.459432
39102,9210530975708218054,8526042588044002101,1.000000
39103,9210530975708218054,8856169137131817223,1.000000
39104,9210530975708218054,8869347744613364434,1.000000


# Items Profile

using TFIDF and cosine similarity for items feature engineering

In [26]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
stopwords_list = stopwords.words('english') + stopwords.words('portuguese') 

vectorizer = TfidfVectorizer(analyzer = 'word', 
                            ngram_range = (1,2),
                            min_df = 0.003,
                            max_df = 0.5,
                            max_features = 5000,
                            stop_words = stopwords_list)

In [33]:
tfidf_matrix = vectorizer.fit_transform(cleaned_article['title'] + "" + cleaned_article['text'])

In [35]:
tfidf_matrix.shape

(3047, 5000)

In [36]:
items_ids = cleaned_article['contentId'].tolist()